In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Merge, Reshape, Dropout
from keras.layers.embeddings import Embedding
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
np.random.seed(10)
tf.set_random_seed(15)

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12458255665286081776
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 457719451091355532
physical_device_desc: "device: XLA_CPU device"
]


In [5]:
def gini(actual, pred):
    assert (len(actual) == len(pred))
    all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
    all = all[np.lexsort((all[:, 2], -1 * all[:, 1]))]
    totalLosses = all[:, 0].sum()
    giniSum = all[:, 0].cumsum().sum() / totalLosses

    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)


def gini_normalized(actual, pred):
    return gini(actual, pred) / gini(actual, actual)

#### Load data 

In [6]:
data = pd.read_csv("../data/train.csv")
calc_cols = data.columns[data.columns.str.startswith('ps_calc_')]
data = data.drop(["id"] + calc_cols.tolist(), axis=1)
X, y = data.loc[:, data.columns != "target"], data.target

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

/usr/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


#### Pre-Processing 

In [8]:
cat_cols = X.columns[X.columns.str.endswith("_cat")]
embed_cols = [c for c in cat_cols if (X[c].nunique(dropna=False) > 2)]
not_embed_cols = [c for c in X.columns if c not in embed_cols]

In [9]:
for c in embed_cols:
    le = LabelEncoder()
    le.fit(X[c])
    X_train.loc[:, c] = le.transform(X_train[c])
    X_test.loc[:, c] = le.transform(X_test[c])

/usr/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [10]:
def get_input_list(X, embed_cols, not_embed_cols):
    input_list = []
    for c in embed_cols:
        input_list.append(X.loc[:, c].values)
    
    input_list.append(X.loc[:, not_embed_cols].values)
    return input_list

#### Model 

In [11]:
def build_model():
    models = []
    
    model_ps_ind_02_cat = Sequential()
    model_ps_ind_02_cat.add(Embedding(5, 3, input_length=1))
    model_ps_ind_02_cat.add(Reshape(target_shape=(3,)))
    models.append(model_ps_ind_02_cat)
    
    model_ps_ind_04_cat = Sequential()
    model_ps_ind_04_cat.add(Embedding(3, 2, input_length=1))
    model_ps_ind_04_cat.add(Reshape(target_shape=(2,)))
    models.append(model_ps_ind_04_cat)
    
    model_ps_ind_05_cat = Sequential()
    model_ps_ind_05_cat.add(Embedding(8, 5, input_length=1))
    model_ps_ind_05_cat.add(Reshape(target_shape=(5,)))
    models.append(model_ps_ind_05_cat)
    
    model_ps_car_01_cat = Sequential()
    model_ps_car_01_cat.add(Embedding(13, 7, input_length=1))
    model_ps_car_01_cat.add(Reshape(target_shape=(7,)))
    models.append(model_ps_car_01_cat)
    
    model_ps_car_02_cat = Sequential()
    model_ps_car_02_cat.add(Embedding(3, 2, input_length=1))
    model_ps_car_02_cat.add(Reshape(target_shape=(2,)))
    models.append(model_ps_car_02_cat)
    
    model_ps_car_03_cat = Sequential()
    model_ps_car_03_cat.add(Embedding(3, 2, input_length=1))
    model_ps_car_03_cat.add(Reshape(target_shape=(2,)))
    models.append(model_ps_car_03_cat)
    
    model_ps_car_04_cat = Sequential()
    model_ps_car_04_cat.add(Embedding(10, 5, input_length=1))
    model_ps_car_04_cat.add(Reshape(target_shape=(5,)))
    models.append(model_ps_car_04_cat)
    
    model_ps_car_05_cat = Sequential()
    model_ps_car_05_cat.add(Embedding(3, 2, input_length=1))
    model_ps_car_05_cat.add(Reshape(target_shape=(2,)))
    models.append(model_ps_car_05_cat)
    
    model_ps_car_06_cat = Sequential()
    model_ps_car_06_cat.add(Embedding(18, 8, input_length=1))
    model_ps_car_06_cat.add(Reshape(target_shape=(8,)))
    models.append(model_ps_car_06_cat)
    
    model_ps_car_07_cat = Sequential()
    model_ps_car_07_cat.add(Embedding(3, 2, input_length=1))
    model_ps_car_07_cat.add(Reshape(target_shape=(2,)))
    models.append(model_ps_car_07_cat)
    
    model_ps_car_09_cat = Sequential()
    model_ps_car_09_cat.add(Embedding(6, 3, input_length=1))
    model_ps_car_09_cat.add(Reshape(target_shape=(3,)))
    models.append(model_ps_car_09_cat)
    
    model_ps_car_10_cat = Sequential()
    model_ps_car_10_cat.add(Embedding(3, 2, input_length=1))
    model_ps_car_10_cat.add(Reshape(target_shape=(2,)))
    models.append(model_ps_car_10_cat)
    
    model_ps_car_11_cat = Sequential()
    model_ps_car_11_cat.add(Embedding(104, 10, input_length=1))
    model_ps_car_11_cat.add(Reshape(target_shape=(10,)))
    models.append(model_ps_car_11_cat)
    
    model_rest = Sequential()
    model_rest.add(Dense(16, input_dim=24))
    models.append(model_rest)

    model = Sequential()
    model.add(Merge(models, mode='concat'))
    model.add(Dense(80))
    model.add(Activation('relu'))
    model.add(Dropout(.35))
    model.add(Dense(20))
    model.add(Activation('relu'))
    model.add(Dropout(.15))
    model.add(Dense(10))
    model.add(Activation('relu'))
    model.add(Dropout(.15))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam')
    
    return model

In [12]:
n_splits = 8
n_runs = 3
n_epochs = 30
batch_size = 4096

cv_ginis = []
full_val_preds = np.zeros(np.shape(X_train)[0])
y_preds = np.zeros((np.shape(X_test)[0], n_splits))
y_preds_train = np.zeros((np.shape(X_train)[0], n_splits))

proc_X_test = get_input_list(X_test, embed_cols, not_embed_cols)
proc_X_train = get_input_list(X_train, embed_cols, not_embed_cols)
kfold = StratifiedKFold(n_splits=n_splits, random_state=231, shuffle=True)

In [14]:
for k, (idx, val_idx) in enumerate(kfold.split(X_train, y_train)):
    X_fold, X_val = X_train.iloc[idx].copy(), X_train.iloc[val_idx].copy()
    y_fold, y_val = y_train.iloc[idx], y_train.iloc[val_idx]
    
    # Upsample
    #  https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283
    pos = (y_fold == 1)
    X_fold = pd.concat([X_fold, X_fold.loc[pos, :]], axis=0)
    y_fold = pd.concat([y_fold, y_fold.loc[pos]], axis=0)
    
    # Shuffle
    shuffled_idx = np.arange(len(X_fold))
    np.random.shuffle(shuffled_idx)
    X_fold = X_fold.iloc[shuffled_idx]
    y_fold = y_fold.iloc[shuffled_idx]
    
    # Preprocessing
    proc_X_fold = get_input_list(X_fold, embed_cols, not_embed_cols)
    proc_X_val = get_input_list(X_val, embed_cols, not_embed_cols)
    
    # Run several times for this fold and average the predictions
    val_preds = 0
    for r in range(n_runs):
        print(r)
        model = build_model()
        model.fit(proc_X_fold, y_fold.values, epochs=n_epochs, batch_size=batch_size, verbose=True)
        val_preds += model.predict(proc_X_val)[:, 0] / n_runs
        
        y_preds[:, k] += model.predict(proc_X_test)[:, 0] / n_runs
        y_preds_train[:, k] += model.predict(proc_X_train)[:, 0] / n_runs
    full_val_preds[val_idx] += val_preds
    
    # Save results on the validation set
    cv_gini = gini_normalized(y_val.values, val_preds)
    cv_ginis.append(cv_gini)
    print('\nFold %i prediction cv gini: %.5f\n' %(k, cv_gini))
    
print('Mean out of fold gini: %.5f' % np.mean(cv_ginis))
print('Full validation gini: %.5f' % gini_normalized(y_train.values, full_val_preds))

0


/usr/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:539: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)
/usr/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:539: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)
/usr/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:539: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)
/usr/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:539: DeprecationWarning: The binary mode of fromstring is

Instructions for updating:
keep_dims is deprecated, use keepdims instead


/usr/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/usr/lib/python3.6/site-packages/_pytest/fixtures.py:834: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  params = attr.ib(convert=attr.converters.optional(tuple))
/usr/lib/python3.6/site-packages/_pytest/fixtures.py:836: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  ids = attr.ib(default=None, convert=_ensure_immutable_ids)
/usr/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/usr/lib/python3.6/site-packa

/usr/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:539: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)
/usr/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:539: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)
/usr/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:539: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)
/usr/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:539: DeprecationWarning: The binary mode of fromstring is

...

/usr/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:539: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)
/usr/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:539: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)
/usr/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:539: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)
/usr/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:539: DeprecationWarning: The binary mode of fromstring is

/usr/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:539: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)
/usr/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:539: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)
/usr/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/usr/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfull

/usr/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/usr/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/usr/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/usr/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


/usr/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:539: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)
/usr/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:539: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)
/usr/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:539: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)
/usr/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:539: DeprecationWarning: The binary mode of fromstring is

Epoch 1/30
377789/377789 [==============================] - 5s 14us/step - loss: 0.3029
Epoch 2/30
377789/377789 [==============================] - 3s 7us/step - loss: 0.2701
Epoch 3/30
377789/377789 [==============================] - 3s 8us/step - loss: 0.2631
Epoch 4/30
377789/377789 [==============================] - 3s 7us/step - loss: 0.2603
Epoch 5/30
377789/377789 [==============================] - 3s 7us/step - loss: 0.2569
Epoch 6/30
377789/377789 [==============================] - 3s 7us/step - loss: 0.2548
Epoch 7/30
377789/377789 [==============================] - 3s 7us/step - loss: 0.2539
Epoch 8/30
377789/377789 [==============================] - 3s 7us/step - loss: 0.2530
Epoch 9/30
377789/377789 [==============================] - 3s 7us/step - loss: 0.2522
Epoch 10/30
377789/377789 [==============================] - 3s 7us/step - loss: 0.2516
Epoch 11/30
377789/377789 [==============================] - 3s 7us/step - loss: 0.2510
Epoch 12/30
377789/377789 [=============

In [15]:
y_pred_final = np.mean(y_preds, axis=1)
y_pred_train_final = np.mean(y_preds_train, axis=1)
gini_normalized(y_test.values, y_pred_final)

0.2752020265510424

#### Gradient Boosting 

In [16]:
for col in X.columns[X.columns.str.endswith("cat")]:
    X.loc[:, col] = X.loc[:, col].astype("category")

/usr/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [17]:
X = pd.get_dummies(X)

In [18]:
X_train = X.loc[X_train.index, :]
X_test = X.loc[X_test.index, :]

In [19]:
float_cols = X_train.columns[(~X_train.columns.str.contains("cat")) & (~X_train.columns.str.contains("bin"))]
X_train[float_cols + "_median"] = (X_train[float_cols] > X_train[float_cols].median()).astype("float")
X_test[float_cols + "_median"] = (X_test[float_cols] > X_test[float_cols].median()).astype("float")
X_train[float_cols + "_mean"] = (X_train[float_cols] > X_train[float_cols].mean()).astype("float")
X_test[float_cols + "_mean"] = (X_test[float_cols] > X_test[float_cols].mean()).astype("float")

In [36]:
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred_xgb = model.predict_proba(X_test)[:, 1]
y_pred_train_xgb = model.predict_proba(X_train)[:, 1]
mean_gini = gini_normalized(y_test, y_pred_xgb)
print(mean_gini)

0.27215442767656517


In [39]:
#model = GradientBoostingClassifier()
#model.fit(X_train.fillna(X_train.median()), y_train)
#y_pred_gb = model.predict_proba(X_test.fillna(X_test.median()))[:, 1]
y_pred_train_gb = model.predict_proba(X_train.fillna(X_train.median()))[:, 1]
mean_gini = gini_normalized(y_test, y_pred_gb)
print(mean_gini)

0.2727717833395658


#### Ensemble 

In [40]:
scores_train = pd.DataFrame({"neural_network": y_pred_train_final, "xgb": y_pred_train_xgb, "gradient_boosting": y_pred_train_gb})

In [41]:
scores_test = pd.DataFrame({"neural_network": y_pred_final, "xgb": y_pred_xgb, "gradient_boosting": y_pred_gb})

In [81]:
y_pred_ensemble = (scores_test * [0.4, 0.4, 0.2]).sum(axis=1)

In [82]:
ensemble_gini = gini_normalized(y_test, y_pred_ensemble)
print(ensemble_gini)

0.2803826594233356
